In [1]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Specify YOUR working directory:
main_dir = 'YOUR_MAIN_DIR'

# Setup:

In [3]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [4]:
!nvidia-smi

Wed Mar  2 16:34:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 57.8 MB/s 
     |████████████████████████████████| 144 kB 87.1 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1b0ef8188f90ba52a92766d784e042b0c4b251f326a8de11a75850760a0bc797
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Installing Packages

In [6]:
import os
os.chdir(main_dir)

!pip install './install/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install './install/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install './install/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl' --no-deps

!pip install './install/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install './install/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install './install/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install './install/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install './install/mmcv_full-1.3.17-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install './install/pycocotools-2.0.2' --no-deps
!pip install './install/mmpycocotools-12.0.3' --no-deps

os.chdir(main_dir + '/install/')
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .
import mmseg


os.chdir(main_dir)
%cd ./install/mmdetection-2.18.0
!pip install -e .
import mmdet

!pip install albumentations==0.4.6

os.chdir(main_dir)

Processing ./install/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
Processing ./install/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
Processing ./install/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
Processing ./install/addict-2.4.0-py3-none-any.whl
Processing ./install/yapf-0.31.0-py2.py3-none-any.whl
Processing ./install/terminal-0.4.0-py3-none-any.whl
Processing ./install/terminaltables-3.1.0-py3-none-any.whl
Processing ./install/

In [7]:
import torch
print(torch.__version__)
print("mmcv_full:")
!pip show mmcv_full |grep Version
print("mmdet:")
!pip show mmdet |grep Version
print("mmseg:")
print(mmseg.__version__)

1.7.1+cu110
mmcv_full:
Version: 1.3.17
mmdet:
Version: 2.18.0
mmseg:
0.21.1


# Importations

In [8]:
import sys
sys.path.append(main_dir + '/install/')
import seg_swin_cell_modules
import cv2
cv2.setNumThreads(0)

import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import init_random_seed, set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

# Helper functions:

In [9]:
def train_seg(config_path):
    cfg = Config.fromfile(config_path)

    ################
    seed_num = 0
    deterministic = True  
    no_validate = False
    distributed = False
    ################


    # create work_dir
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    # dump config
    cfg.dump(osp.join(cfg.work_dir, osp.basename(config_path)))
    # init the logger before other steps
    timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
    logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

    # init the meta dict to record some important information such as
    # environment info and seed, which will be logged
    meta = dict()
    # log env info
    env_info_dict = collect_env()
    env_info = '\n'.join([f'{k}: {v}' for k, v in env_info_dict.items()])
    dash_line = '-' * 60 + '\n'
    logger.info('Environment info:\n' + dash_line + env_info + '\n' +
                dash_line)

    # log some basic info
    logger.info(f'Distributed training: {distributed}')
    logger.info(f'Config:\n{cfg.pretty_text}')

    # set random seeds
    seed = init_random_seed(seed_num)
    logger.info(f'Set random seed to {seed}, '
                f'deterministic: {deterministic}')
    set_random_seed(seed, deterministic=deterministic)
    cfg.seed = seed
    meta['seed'] = seed
    meta['exp_name'] = osp.basename(config_path)

    model = build_segmentor(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))
    model.init_weights()

    # SyncBN is not support for DP
    if not distributed:
        warnings.warn(
            'SyncBN is only supported with DDP. To be compatible with DP, '
            'we convert SyncBN to BN. Please use dist_train.sh which can '
            'avoid this error.')
        model = revert_sync_batchnorm(model)

    logger.info(model)

    datasets = [build_dataset(cfg.data.train)]
    if len(cfg.workflow) == 2:
        val_dataset = copy.deepcopy(cfg.data.val)
        val_dataset.pipeline = cfg.data.train.pipeline
        datasets.append(build_dataset(val_dataset))
    if cfg.checkpoint_config is not None:
        # save mmseg version, config file content and class names in
        # checkpoints as meta data
        cfg.checkpoint_config.meta = dict(
            mmseg_version=f'{__version__}+{get_git_hash()[:7]}',
            config=cfg.pretty_text,
            CLASSES=datasets[0].CLASSES,
            PALETTE=datasets[0].PALETTE)
    # add an attribute for visualization convenience
    model.CLASSES = datasets[0].CLASSES
    # passing checkpoint meta for saving best checkpoint
    meta.update(cfg.checkpoint_config.meta)
    train_segmentor(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=(not no_validate),
        timestamp=timestamp,
        meta=meta)

# Train:

In [10]:
config_path = main_dir + '/models/seg/seg_upernet_swin/configs/config_seg_upernet_swin.py' 

In [ ]:

train_seg(config_path)
